In [1]:
import os.path
import mne
import numpy as np
import pandas as pd

from utils.dataloader import load_subject_labels

wd = '/Volumes/Guillaume EEG Project'
epoch_data_path = os.path.join(wd, 'Berlin_Data/EEG/preprocessed/stim_epochs_incl_response')
raw_data_path = os.path.join(wd, 'Berlin_Data/EEG/raw')

In [2]:
# Load the raw data
subject_id = 13

raws = []
for file in [os.path.join(wd, raw_data_path, '{}/{}{}.vhdr'.format(subject_id, subject_id, suffix)) for suffix in ['', '_2', '_3', '_4', '_5']]:
    if os.path.isfile(file):
        raw = mne.io.read_raw_brainvision(file)
        raws.append(raw)

behav_data = load_subject_labels(os.path.join(wd, raw_data_path), subject_id=subject_id)

print(behav_data)

Extracting parameters from /Volumes/Guillaume EEG Project/Berlin_Data/EEG/raw/13/13.vhdr...
Setting channel info structure...
     response  confidence  choice_rt  correct  session  run
0         1.0         1.0   1.170349      1.0        1    2
1         1.0         1.0   1.472831      1.0        1    2
2         1.0         1.0   0.999485      1.0        1    2
3         1.0         1.0   1.568129      1.0        1    2
4        -1.0         2.0   0.450765      1.0        1    2
..        ...         ...        ...      ...      ...  ...
325      -1.0         2.0   0.463984      0.0        1    4
326       1.0         1.0   1.783581      1.0        1    4
327      -1.0         2.0   0.156226      0.0        1    4
328       NaN         NaN        NaN      NaN        1    4
329      -1.0         2.0   0.409565      0.0        1    4

[330 rows x 6 columns]


In [3]:
## SPLIT RAW INTO BLOCKS
# Specify the annotation type you want to use for splitting
split_annotation_type = 'Stimulus/S105'

raw_segments = []
for raw in raws:
    # Find the onset times of the relevant annotations
    split_onsets = [annot['onset'] for annot in raw.annotations if annot['description'] == split_annotation_type]

    # Sort the onsets and add the start and end times of the raw object for splitting
    split_onsets = split_onsets + [raw.times[-1]]

    # Iterate over consecutive pairs of split points to create new raw segments
    for start, end in zip(split_onsets[:-1], split_onsets[1:]):
        raw_segment = raw.copy().crop(tmin=start, tmax=end, include_tmax=False)
        raw_segments.append(raw_segment)

# Now raw_segments is a list of mne.io.Raw objects split at the specified annotations
print(len(raw_segments))

for seg in raw_segments:
    print(seg)
    # seg_anno_vc = pd.Series(seg.annotations.description).value_counts()
    # print(seg.info)
    # print(seg_anno_vc)
    # print('')

7
<RawBrainVision | 13.eeg, 63 x 318269 (318.3 s), ~93 kB, data not loaded>
<RawBrainVision | 13.eeg, 63 x 486684 (486.7 s), ~93 kB, data not loaded>
<RawBrainVision | 13.eeg, 63 x 500206 (500.2 s), ~93 kB, data not loaded>
<RawBrainVision | 13.eeg, 63 x 508036 (508.0 s), ~93 kB, data not loaded>
<RawBrainVision | 13.eeg, 63 x 464151 (464.1 s), ~93 kB, data not loaded>
<RawBrainVision | 13.eeg, 63 x 462940 (462.9 s), ~93 kB, data not loaded>
<RawBrainVision | 13.eeg, 63 x 414082 (414.1 s), ~93 kB, data not loaded>


In [4]:
# TODO: select correct segments
raw_segments = raw_segments[1:]
print(len(raw_segments))

6


In [5]:
## CREATE NEW ANNOTATIONS FOR RESPONSE TIMES
for i, seg in enumerate(raw_segments):
    df = behav_data[behav_data['run'] == i + 1]

    # Step 1: Extract the onsets of 'Stimulus/S 48' annotations
    stimulus_onsets = [annot['onset'] for annot in seg.annotations if annot['description'] == 'Stimulus/S 48']

    # Step 2: Ensure the number of onsets matches the number of rows in the DataFrame
    if len(stimulus_onsets) != len(df):
        raise ValueError("The number of 'Stimulus/S 48' annotations does not match the number of rows in the DataFrame.")

    # Step 3: Create new annotations based on the reaction times in the DataFrame
    for onset, choice_rt in zip(stimulus_onsets, df['choice_rt']):
        if i == 5:
            pass
        if not np.isnan(choice_rt):
            new_onset = onset + choice_rt  # Calculate the new annotation onset time
            seg.annotations.append(onset=new_onset, duration=0, description='Response')
        else:
            new_onset = onset + 0.001  # Calculate the new annotation onset time
            seg.annotations.append(onset=new_onset, duration=0, description='No Response')

In [6]:
# for i, an in enumerate(raw_segments[-1].annotations):
#     if an['description'] in [*['Stimulus/S{:>3}'.format(stim) for stim in [20, 21, 22, 23, 48]], 'Response', 'No Response']:
#         print(an)
#     if i == 200:
#         break

In [ ]:
## EXTRACT EPOCHS
# Define the epochs
tmin = -2.25  # Start of each epoch (e.g., 2250 ms before the event)
tmax = 0.25   # End of each epoch (e.g., 250 ms after the event)

epochs_list = []
for seg in raw_segments:
    events, event_id = mne.events_from_annotations(seg, event_id={'Response': 1, 'No Response': 2})
    epochs_list.append(mne.Epochs(seg, events, event_id=event_id, tmin=tmin, tmax=tmax, baseline=None, preload=True))

# Step 5: Inspect the epochs
print(epochs_list[0])

Used Annotations descriptions: [np.str_('Response')]
Not setting metadata
53 matching events found
No baseline correction applied
0 projection items activated
Loading data for 53 events and 2501 original time points ...
0 bad epochs dropped
Used Annotations descriptions: [np.str_('Response')]
Not setting metadata
55 matching events found
No baseline correction applied
0 projection items activated
Loading data for 55 events and 2501 original time points ...
0 bad epochs dropped
Used Annotations descriptions: [np.str_('Response')]
Not setting metadata
50 matching events found
No baseline correction applied
0 projection items activated
Loading data for 50 events and 2501 original time points ...
0 bad epochs dropped
Used Annotations descriptions: [np.str_('Response')]
Not setting metadata
43 matching events found
No baseline correction applied
0 projection items activated
Loading data for 43 events and 2501 original time points ...
0 bad epochs dropped
Used Annotations descriptions: [np.s

In [ ]:
# TODO: select epochs that have response, drop the rest according to annotation 'No Response'